In [1]:
import subprocess
import pandas as pd
import os
import glob
from tqdm import tqdm
from os.path import basename as bn, join, split as sp
import parselmouth
import numpy as np
from parselmouth.praat import call
from scipy.io.wavfile import write
import praat_formants_python as pfp


# DATASET Path and Constants

In [2]:
# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Synth Vowel info Export CSV filename
SYNTH_VOWEL_INFO_IMP_FILENAME = "d_base_formants_synth_vowels_vowlimLIM.csv"
SYNTH_VOWEL_INFO_IMP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_INFO_IMP_FILENAME)

# Vowel subset Export CSV filename
SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME = "e_synth-vowels_formant_estimation_vowlimLIM.csv"
SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME)


### Import Synth Vowel (BASE) Formant Info dataframe

In [3]:
VOWEL_LIMIT = 100
SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF = pd.read_csv(SYNTH_VOWEL_INFO_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF

,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base,synth_vowel_path
0,0,MWGR0,M,0.066062,iy,118.56,495.03,2131.18,2950.80,495.03,2124.55,2957.00,./audio_exports/vowlim100/iy_0_MWGR0_M_119.wav
1,1,MJES0,M,0.074375,iy,112.66,316.93,2098.74,2844.58,325.92,2098.60,2876.64,./audio_exports/vowlim100/iy_1_MJES0_M_113.wav
2,2,MSTK0,M,0.077688,iy,150.65,426.11,1931.06,2447.48,420.31,2046.93,2372.78,./audio_exports/vowlim100/iy_2_MSTK0_M_151.wav
3,3,MJDM0,M,0.121500,iy,94.78,467.55,1824.06,2588.08,450.22,1824.37,2533.09,./audio_exports/vowlim100/iy_3_MJDM0_M_95.wav
4,4,MLBC0,M,0.086125,iy,107.83,460.67,1942.34,2558.01,461.12,2026.29,2558.57,./audio_exports/vowlim100/iy_4_MLBC0_M_108.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,MRJM3,M,0.085500,ax-h,112.14,743.40,1492.04,2672.56,671.51,1352.75,2550.95,./audio_exports/vowlim100/ax-h_3816_MRJM3_M_11...
3817,3817,MAPV0,M,0.070187,ax-h,133.61,646.22,1847.19,2720.36,613.69,1837.41,2696.50,./audio_exports/vowlim100/ax-h_3817_MAPV0_M_13...
3818,3818,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,1647.24,2806.36,./audio_exports/vowlim100/ax-h_3818_MSFV0_M_10...
3819,3819,FLJG0,F,0.073313,ax-h,195.82,852.51,1813.99,2963.22,861.90,1809.55,2976.29,./audio_exports/vowlim100/ax-h_3819_FLJG0_F_19...


## FUNCTION: Measure formants of synth vowels | PRAAT FORMANTS

In [13]:
pfp.clear_formant_cache()

def measure_pitch(audio_path: str) -> float:
    f0min, f0max = [75, 600]
    
    sound = parselmouth.Sound(audio_path) # read the sound
    pitch = call(sound, "To Pitch", 0, f0min, f0max) #create a praat pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    return mean_pitch
    
def measure_formants_pfp(audio_path: str, start_sec: float, end_sec: float) -> dict:
    formants: list = pfp.formants_at_interval(
        audio_path, start_sec, end_sec, maxformant=5500, winlen=1, preemph=50
    )

    pitch_mean = measure_pitch(audio_path)
    pitch_mean = np.round(pitch_mean, 2)
    
    formants_mean = formants.mean(axis=0)
    formants_mean = list(formants_mean)[1:]  # skip time
    formants_mean = np.round(formants_mean, 2)  # round

    formants_median = np.median(formants, axis=0)
    formants_median = list(formants_median)[1:]  # skip time
    formants_median = np.round(formants_median, 2) # round


    return {
        "pitch_mean_synthvow_praat": pitch_mean,

        "F1_mean_synthvow_praat": formants_mean[0],
        "F2_mean_synthvow_praat": formants_mean[1],
        "F3_mean_synthvow_praat": formants_mean[2],

        "F1_median_synthvow_praat": formants_median[0],
        "F2_median_synthvow_praat": formants_median[1],
        "F3_median_synthvow_praat": formants_median[2],
    }


## FUNCTION: Measure formants of synth vowels | DEEPFORMANTS

In [5]:
def get_deepformants(audio_file: str, begin: float, end: float):
    proc = subprocess.check_output('/home/jeevan/dev/anaconda3/envs/pytorch/bin/python formants.py ' + audio_file + ' /home/jeevan/Desktop/DeepFormants/test.csv --begin ' +  str(begin) + ' --end ' + str(end), cwd="/home/jeevan/Desktop/DeepFormants/", shell=True)
    df = pd.read_csv("/home/jeevan/Desktop/DeepFormants/test.csv")
    return df.iloc[0,1:].tolist()
   

def measure_formants_df(audio_path: str,  start_sec: float, end_sec: float):
    _formants: list = get_deepformants(audio_path, start_sec, end_sec)
    _formants = np.round(_formants, 2)  # round

    return {
        "F1_mean_synthvow_deepformant": _formants[0],
        "F2_mean_synthvow_deepformant": _formants[1],
        "F3_mean_synthvow_deepformant": _formants[2],
        "F4_mean_synthvow_deepformant": _formants[3],
    }


### FUNCTION TESTS

In [22]:
v, audf = SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF.loc[np.random.randint(0, len(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF)), ["vowel_name", "synth_vowel_path"]]
dur = 1.7/2
start, end = (dur - 0.5), (dur + 0.5)
audf = os.path.realpath(audf)
print(audf, start, end )

print(measure_formants_pfp(audf, start, end))
# print(measure_formants_df(audf, start, end))


/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim100/ao_1221_MDLS0_M_141.wav 0.35 1.35
{'pitch_mean_synthvow_praat': 140.95, 'F1_mean_synthvow_praat': 668.5, 'F2_mean_synthvow_praat': 837.95, 'F3_mean_synthvow_praat': 1101.86, 'F1_median_synthvow_praat': 668.5, 'F2_median_synthvow_praat': 837.95, 'F3_median_synthvow_praat': 1101.86}


## FUNCTION: Synth Vowel PITCH, FORMANT Estimation | Praat, DeepFormants, +

In [29]:
def estimate_synth_vowel_formants(vowel_info):
    audio_file = vowel_info["synth_vowel_path"]
    audio_file = os.path.realpath(audio_file)
    vowel_name = vowel_info["vowel_name"]

    mdur = 1.7/2
    start_sec, end_sec = (mdur - 0.5), (mdur + 0.5)
    try :
        formant_estimates_pfp = measure_formants_pfp(audio_file, start_sec, end_sec) # estimator 1
        # formant_estimates_df = measure_formants_df(audio_file, start_sec, end_sec) # estimator 2
        # TODO: estimators 3, 4, 5

        new_vowel_info = dict(vowel_info) | formant_estimates_pfp #| formant_estimates_df
    except:
        print(audio_file)
    
    return new_vowel_info
    

syvinfo = SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF.loc[np.random.randint(0, len(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF))]
estimate_synth_vowel_formants(syvinfo)

{'index': 2959,
 'person_id': 'FCEG0',
 'sex': 'F',
 'duration_second': 0.065,
 'vowel_name': 'ux',
 'pitch_mean_praat_base': 244.59,
 'F1_mean_praat_base': 421.7,
 'F2_mean_praat_base': 2243.51,
 'F3_mean_praat_base': 2840.15,
 'F1_median_praat_base': 449.27,
 'F2_median_praat_base': 2375.2,
 'F3_median_praat_base': 2888.42,
 'synth_vowel_path': './audio_exports/vowlim100/ux_2959_FCEG0_F_245.wav',
 'pitch_mean_synthvow_praat': 244.93,
 'F1_mean_synthvow_praat': 463.1,
 'F2_mean_synthvow_praat': 763.52,
 'F3_mean_synthvow_praat': 2213.09,
 'F1_median_synthvow_praat': 463.1,
 'F2_median_synthvow_praat': 763.52,
 'F3_median_synthvow_praat': 2213.09}

### Create and export Synth Vowel Formant Estimation Result dataframe

In [31]:

SYNTH_VOWELS_FORMANT_DF = pd.DataFrame([estimate_synth_vowel_formants(syv_i) for _, syv_i in tqdm(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF[0:].iterrows())])

SYNTH_VOWELS_FORMANT_DF.columns = SYNTH_VOWELS_FORMANT_DF.columns.str.replace("synthvow_praat", "synthvow_praat")

csv_path = SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}_fullwin")
print(csv_path)

if not os.path.exists(csv_path):
    SYNTH_VOWELS_FORMANT_DF.to_csv(csv_path, index=False)

SYNTH_VOWELS_FORMANT_DF

./exports/e_synth-vowels_formant_estimation_vowlim100_fullwin.csv


,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base,synth_vowel_path,pitch_mean_synthvow_filter_praat,F1_mean_synthvow_filter_praat,F2_mean_synthvow_filter_praat,F3_mean_synthvow_filter_praat,F1_median_synthvow_filter_praat,F2_median_synthvow_filter_praat,F3_median_synthvow_filter_praat
0,0,MWGR0,M,0.066062,iy,118.56,495.03,2131.18,2950.80,495.03,2124.55,2957.00,./audio_exports/vowlim100/iy_0_MWGR0_M_119.wav,118.97,476.22,996.44,2131.14,476.22,996.44,2131.14
1,1,MJES0,M,0.074375,iy,112.66,316.93,2098.74,2844.58,325.92,2098.60,2876.64,./audio_exports/vowlim100/iy_1_MJES0_M_113.wav,112.97,325.10,951.24,2099.49,325.10,951.24,2099.49
2,2,MSTK0,M,0.077688,iy,150.65,426.11,1931.06,2447.48,420.31,2046.93,2372.78,./audio_exports/vowlim100/iy_2_MSTK0_M_151.wav,151.00,439.52,951.33,1940.93,439.52,951.33,1940.93
3,3,MJDM0,M,0.121500,iy,94.78,467.55,1824.06,2588.08,450.22,1824.37,2533.09,./audio_exports/vowlim100/iy_3_MJDM0_M_95.wav,95.00,468.86,885.35,1820.12,468.86,885.35,1820.12
4,4,MLBC0,M,0.086125,iy,107.83,460.67,1942.34,2558.01,461.12,2026.29,2558.57,./audio_exports/vowlim100/iy_4_MLBC0_M_108.wav,108.00,443.95,955.11,1944.16,443.95,955.11,1944.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,MRJM3,M,0.085500,ax-h,112.14,743.40,1492.04,2672.56,671.51,1352.75,2550.95,./audio_exports/vowlim100/ax-h_3816_MRJM3_M_11...,111.97,741.25,931.60,1494.29,741.25,931.60,1494.29
3817,3817,MAPV0,M,0.070187,ax-h,133.61,646.22,1847.19,2720.36,613.69,1837.41,2696.50,./audio_exports/vowlim100/ax-h_3817_MAPV0_M_13...,134.01,655.39,879.24,1852.45,655.39,879.24,1852.45
3818,3818,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,1647.24,2806.36,./audio_exports/vowlim100/ax-h_3818_MSFV0_M_10...,99.99,765.81,900.97,1880.47,765.81,900.97,1880.47
3819,3819,FLJG0,F,0.073313,ax-h,195.82,852.51,1813.99,2963.22,861.90,1809.55,2976.29,./audio_exports/vowlim100/ax-h_3819_FLJG0_F_19...,196.08,804.76,1080.45,1795.72,804.76,1080.45,1795.72


In [ ]:
SYNTH_VOWELS_FORMANT_DF.describe()

,index,duration_second,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base,pitch_mean_synthvow_praat,F1_mean_synthvow_praat,F2_mean_synthvow_praat,F3_mean_synthvow_praat,F1_median_synthvow_praat,F2_median_synthvow_praat,F3_median_synthvow_praat
count,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3820.000000,3597.000000,3499.000000,3820.000000,3597.000000,3499.000000
mean,1910.342932,0.116878,164.526160,610.366908,1596.674411,2614.195513,609.815838,1586.919398,2604.130626,179.393008,774.299003,1465.651145,2185.020600,787.602709,1485.808916,2189.655513
std,1103.112728,0.045308,44.309751,128.553596,355.077225,324.400737,140.812121,376.672965,350.911751,56.993220,659.428805,440.781656,447.999061,692.261111,471.119556,497.166925
min,0.000000,0.064000,82.940000,255.420000,747.820000,1433.480000,259.070000,657.220000,1382.010000,76.820000,154.950000,579.570000,1314.400000,140.080000,575.050000,1057.940000
25%,955.750000,0.081875,124.315000,521.852500,1335.360000,2430.797500,512.485000,1310.682500,2416.345000,138.040000,447.740000,1223.160000,1943.920000,437.447500,1213.570000,1910.800000
50%,1910.500000,0.105000,162.580000,603.110000,1567.530000,2630.845000,600.915000,1556.210000,2616.755000,179.860000,593.315000,1377.130000,2121.540000,596.815000,1395.340000,2124.210000
75%,2865.250000,0.140203,202.307500,686.360000,1822.510000,2825.890000,690.920000,1815.525000,2831.732500,207.995000,755.857500,1561.790000,2307.415000,769.667500,1612.570000,2350.625000
max,3820.000000,0.424375,290.680000,1854.180000,2874.720000,3842.220000,2095.860000,2891.460000,4034.360000,590.330000,3866.980000,3688.530000,4380.550000,4023.190000,3519.910000,4422.350000


# TEST

In [ ]:
# :)

